**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Obtenga las letras asociadas a cada clave para el siguiente archivo.

In [1]:
%%writefile input.txt
0 	 C,F,A,B,D,I,H
1 	 A,H,C,I,F,D,B
2 	 B,H,I,F,G
3 	 C,B,D
4 	 D,C,I,G,H
5 	 B,D,C,H,A
6 	 H,D,C,B,G,F,D
7 	 F,A,G,C,B,D,H,I
8 	 F,A,I,B,D
9 	 F,A,B,D,C,D,G,I
10 	 D,B,A,C,H
11 	 G,D,B,H,I,C,F
12 	 D,D,C,F,B,A,H,G
13 	 F,A,D
14 	 D,A,C,G
15 	 H,A,F,D,B,C,G,I
16 	 A,I,B,D
17 	 C,B,G,A,D,H,F
18 	 I,B,A,H,D,F
19 	 B,D,F,D,I
20 	 C,B,H,F,I,G,D,D
21 	 F,A,B,C,G,D
22 	 I,G,F,C,H,B
23 	 H,F,C,B,D,D,A
24 	 F,D,G,A,H
25 	 G,H,B,C,A,F,I
26 	 G,F,B,A,H,D,D,I
27 	 B,A,H,I,D,G,F
28 	 A,H,D,F,C
29 	 C,D,A,F,G,B,H,D

Overwriting input.txt


## Mapper

In [2]:
%%writefile mapper.py
#! /usr/bin/env python

import sys

##
## Se usa una clase iterable para implementar el mapper.
##

class Mapper:
    
    def __init__(self, stream):
        ## 
        ## almacena el flujo de entrada como una
        ## variable del objeto
        ##
        self.stream = stream
    
    def emit(self, key, value):
        ##
        ## escribe al flujo estándar de salida
        ##
        sys.stdout.write("{}\t{}\n".format(key, value))
        
        
    def status(self, message):
        ##
        ## imprime un reporte en el flujo de error
        ## no se debe usar el stdout, ya que en este 
        ## unicamente deben ir las parejas (key, value)
        ##
        sys.stderr.write('reporter:status:{}\n'.format(message))

        
    def counter(self, counter, amount=1, group="ApplicationCounter"):
        ## 
        ## imprime el valor del contador
        ##
        sys.stderr.write('reporter:counter:{},{},{}\n'.format(group, counter, amount))
        
    def map(self):

        #word_counter = 0
        for word in self:
            self.emit(key=word[0], value=word[1])

            
    def __iter__(self):
        ##
        ## itera sobre cada linea de código recibida
        ## a través del flujo de entrada
        ##
        try:
            diccionario= dict(line.split() for line in self.stream)
            for item in diccionario:
                pares=diccionario[item].split(',')

                for word in pares:
                    yield word,item
        except:
            pass


if __name__ == "__main__": 
    mapper = Mapper(sys.stdin)
    
    ##
    ## ejecuta el mapper
    ##
    mapper.map()
    pass

Overwriting mapper.py


## Reducer

In [3]:
%%writefile reducer.py
#!/usr/bin/env python

import sys
import itertools

class Reducer:
    
    def __init__(self, stream):
        self.stream = stream
        
    def emit(self, key, value):
        sys.stdout.write("{} {}\n".format(key,value)) 

    def reduce(self):
        ##
        ## Esta función reduce los elementos que 
        ## tienen la misma clave
        ## 

        diccionario={}
        for key, group in itertools.groupby(self, lambda x: x[0]):
            if str(key) not in diccionario:
                diccionario.update({str(key):[]})
            for _,val in group:
                diccionario[key].append(val)
        
        for item in diccionario:
            numeros=''
            for numero in diccionario[item]:
                if numeros =='':
                    numeros=numeros+numero
                else:
                    numeros=numeros+','+numero
            self.emit(key=item, value=numeros)
                
    def __iter__(self):
        
        for line in self.stream:
            ##
            ## Lee el stream de datos y lo parte 
            ## en (clave, valor)
            ##
            key, val= line.split("\t") 
           
            ##
            ## retorna la tupla (clave, valor)
            ## como el siguiente elemento del ciclo for
            ##
            yield key, val.replace('\n','')

if __name__ == '__main__': 
    reducer = Reducer(sys.stdin)
    reducer.reduce()
    pass

Overwriting reducer.py


## Ejecución

In [4]:
%%bash
rm -rf output
STREAM=$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar
chmod +x mapper.py
chmod +x reducer.py
hadoop jar $STREAM -input input.txt -output output  -mapper mapper.py -reducer reducer.py
cat output/part-00000

A 1,18,26,21,10,7,0,12,25,8,27,13,14,15,24,29,16,28,23,9,17,5	
B 22,19,20,21,1,29,2,3,5,6,7,27,8,9,26,11,12,0,25,15,16,17,23,18,10	
C 0,29,28,25,23,22,21,20,17,15,14,12,11,10,9,7,6,5,4,3,1	
D 19,19,18,17,23,23,1,16,24,15,14,13,29,12,12,11,10,9,9,26,26,8,7,27,6,0,6,5,4,3,28,21,20,20,29	
F 18,25,6,20,9,0,1,23,11,8,24,27,28,17,21,19,13,29,26,12,22,7,15,2	
G 22,6,27,26,11,25,2,9,20,24,14,21,15,12,29,17,7,4	
H 25,12,1,11,10,26,0,2,20,27,7,22,6,18,23,17,5,28,4,24,15,29	
I 7,16,25,15,8,26,2,9,11,4,22,18,20,0,27,19,1	


In [ ]:
!rm -rf mapper.py reducer.py output input.txt

---

Para realizar la evaluación automática de este libro:

* Abra un Terminal.
* Asegurece que esat en la misma carpeta que contiene este notebook.
* Salve el notebook.
* Ejecute el siguiente comando:

      ./gradetool 11-Taller.ipynb

---